### The first part of this notebook replicates the "filter_sentence" notebook to get the dataset ready for setences selection. New implementations are in "Test one row" and "The real function" section

### Filter sentences (copy)

In [1]:
!pip install datasets
!pip install transformers
!pip install rouge
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.3 MB/s 
     |████████████████████████████████| 163 kB 46.3 MB/s 
     |████████████████████████████████| 212 kB 48.2 MB/s 
     |████████████████████████████████| 95 kB 2.1 MB/s 
     |████████████████████████████████| 115 kB 34.0 MB/s 
     |████████████████████████████████| 127 kB 13.3 MB/s 
     |████████████████████████████████| 115 kB 43.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████|

In [84]:
from datasets import load_dataset
import re
# from nltk.tokenize import sent_tokenize
# import nltk
import numpy as np
import tqdm
import spacy
# nltk.download('punkt')

In [3]:
from datasets import load_dataset
dataset = load_dataset('multi_news', split='train')

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [4]:
!pip install rouge_score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4abfde88ddb215a7ee2ad82717820f86f297b94b35f1c4c012485f8c941e44e8
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [5]:
!python -m spacy download en_core_web_sm

2022-11-07 20:53:11.294836: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
nlp = spacy.load("en_core_web_sm")
#nlp(small_dataset['document'][0])

In [7]:
def score_sentence(document):
    text = document['document']
    doc = nlp(text)
    #print('1')
    rouge_scores = []
    entity_counts = []
    
    
    for s in doc.sents:
        target = text[s.start_char:s.end_char]
        rest_doc = text[:s.start_char] + text[s.end_char:]
#         print(target)
        try:
            score = scorer.score(target,
                      rest_doc)['rouge1'][2]
            rouge_scores.append(score)
        except:
            rouge_scores.append(0)
        

        entity_counts.append(len(nlp(str(s)).ents))   
            
    document['rouge_scores'] = rouge_scores
    document['entity_counts'] = entity_counts
    
#     rank = (-np.array(scores)).argsort()
#     threshold = int(len(rank)*0.8)
#     rank = rank[:threshold]
#     rank.sort()
#     filtered_doc = ' '.join([splited_document[i] for i in rank])
#     doc['document'] = filtered_doc
    return document

In [30]:
small_dataset = dataset.select(list(range(5)))

small_dataset = small_dataset.add_column("rouge_scores", [[0]] * len(small_dataset))
small_dataset = small_dataset.add_column("entity_counts", [[0]] * len(small_dataset))
small_dataset = small_dataset.add_column("filter_sentences", [[0]] * len(small_dataset))

In [31]:
small_dataset

Dataset({
    features: ['document', 'summary', 'rouge_scores', 'entity_counts', 'filter_sentences'],
    num_rows: 5
})

In [32]:
#import time 
#start  = time.time()
d = small_dataset.map(score_sentence, num_proc = 5)
#end = time.time()
#print (end-start)

#0:   0%|          | 0/1 [00:00<?, ?ex/s]

#1:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

#4:   0%|          | 0/1 [00:00<?, ?ex/s]

In [38]:
d['rouge_scores']

[[0.0,
  0.027027027027027025,
  0.09459459459459459,
  0.08108108108108109,
  0.08108108108108109,
  0.0945945945945946,
  0.05405405405405405,
  0.05405405405405406,
  0.020270270270270275,
  0.027027027027027025,
  0.0,
  0.006756756756756757,
  0.08783783783783784,
  0.05405405405405405,
  0.09459459459459461,
  0.05405405405405405,
  0.03378378378378378,
  0.02027027027027027,
  0.0472972972972973,
  0.03378378378378378,
  0.05405405405405406],
 [0.004410143329658215,
  0.07056229327453142,
  0.017640573318632856,
  0.017640573318632856,
  0.0,
  0.05292171995589856,
  0.028665931642778388,
  0.044101433296582136,
  0.024255788313120173,
  0.02866593164277839,
  0.03528114663726571,
  0.09040793825799338,
  0.0,
  0.03528114663726572,
  0.05292171995589856,
  0.013230429988974642,
  0.024255788313120176,
  0.0,
  0.030871003307607493,
  0.061742006615215,
  0.0,
  0.002205071664829107,
  0.07056229327453142,
  0.0,
  0.013230429988974642,
  0.03307607497243661,
  0.026460859977949

### Test on one row

In [133]:
percentage = 0.8 #percentage to keep
top = 5 # keep first t sentences 
row = d['document'][4]
sentences = list(nlp(row).sents)
pointers = [i for i in range(len(sentences)) if '|||||' in sentences[i].text] #seperate articles
scores = d['rouge_scores'][4]
filter = []
score_splits = [sl.tolist()for sl in np.split(scores, pointers)]
for a in range(len(score_splits)):
  rest = score_splits[a][5:]
  n_leave = int(len(rest)*percentage) #leave 0.8 percent of the rest of the sentences (except top sentences)
  index_leave = sorted(range(len(rest)), key = lambda sub: rest[sub])[-n_leave:] #index of top n_leave scores
  if len(score_splits[a])>=top:
    filter.append([1]*top+[0]*len(rest))
  else:
    filter.append([1]*(len(score_splits[a])))
  for i in index_leave:
    filter[a][i+top] = 1
#print(filter)
filter = sum(filter,[]) #merge lists into one list
print(filter)

[[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]]
[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]


### The real function on the dataset

In [119]:
def filter_sentence(document):
  percentage = 0.8 #percentage to keep
  top = 5 # keep first t sentences 
  #row = d['document'][0]
  sentences = list(nlp(document['document']).sents)
  pointers = [i for i in range(len(sentences)) if '|||||' in sentences[i].text] #seperate articles
  scores = document['rouge_scores']
  filter = []
  score_splits = [sl.tolist()for sl in np.split(scores, pointers)]
  for a in range(len(score_splits)):
    rest = score_splits[a][5:]
    n_leave = int(len(rest)*percentage) #leave 0.8 percent of the rest of the sentences (except top sentences)
    index_leave = sorted(range(len(rest)), key = lambda sub: rest[sub])[-n_leave:] #index of top n_leave scores
    if len(score_splits[a])>=top:
      filter.append([1]*top+[0]*len(rest))
    else:
      filter.append([1]*(len(score_splits[a])))
    for i in index_leave:
      filter[a][i+top] = 1
  filter = sum(filter,[])
  document['filter_sentences'] = filter
  return document
  

In [120]:
m = d.map(filter_sentence, num_proc = 5)
m

#1:   0%|          | 0/1 [00:00<?, ?ex/s]

#0:   0%|          | 0/1 [00:00<?, ?ex/s]

#4:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

Dataset({
    features: ['document', 'summary', 'rouge_scores', 'entity_counts', 'filter_sentences'],
    num_rows: 5
})

In [134]:
#%pprint
m['filter_sentences']

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]]